In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index


array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
x_test_adv_0_1.shape

(1025739, 56)

In [6]:
from xgboost import XGBClassifier

print('xgb')
xgb100 = XGBClassifier()
xgb100.fit(x_test_adv_0_1, y_label_0_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf100 = RandomForestClassifier()
rf100.fit(x_test_adv_0_1, y_label_0_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt100 = DecisionTreeClassifier()
dt100.fit(x_test_adv_0_1, y_label_0_1)

xgb
RF
DT


DecisionTreeClassifier()

In [7]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [8]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [22326   149  2492   856    66  2019     1   211 85586   201    64]
(22326, 56) (22326,)
Save 0 to baseline_Def1.npy
(149, 56) (149,)
Save 1 to baseline_Def2.npy
(2492, 56) (2492,)
Save 2 to baseline_Def3.npy
(856, 56) (856,)
Save 3 to baseline_Def4.npy
(66, 56) (66,)
Save 4 to baseline_Def5.npy
(2019, 56) (2019,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(211, 56) (211,)
Save 7 to baseline_Def8.npy
(85586, 56) (85586,)
Save 8 to baseline_Def9.npy
(201, 56) (201,)
Save 9 to baseline_Def10.npy
(64, 56) (64,)
Save 10 to baseline_Def11.npy
Execution Time: 0.682233 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [109193    155   1475   1223    451 175150      1    119  54020     26
    100]
(109193, 56) (109193,)
Save 0 to BIM_Def1.npy
(155, 56) (155,)
Save 1 to BIM_Def2.npy
(1475, 56) (1475,)
Save 2 to BIM_Def3.npy
(1223, 56) (1223,)
Save 3 to BIM_Def4.npy
(451, 56) (451,)
Save 4 to BIM_Def5.npy
(175150, 56) (175

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [23219   229  2202  1020   138  3238   322 83258   237   108]
(23219, 56) (23219,)
Save 0 to baseline_Def1.npy
(229, 56) (229,)
Save 1 to baseline_Def2.npy
(2202, 56) (2202,)
Save 2 to baseline_Def3.npy
(1020, 56) (1020,)
Save 3 to baseline_Def4.npy
(138, 56) (138,)
Save 4 to baseline_Def5.npy
(3238, 56) (3238,)
Save 5 to baseline_Def6.npy
(322, 56) (322,)
Save 7 to baseline_Def8.npy
(83258, 56) (83258,)
Save 8 to baseline_Def9.npy
(237, 56) (237,)
Save 9 to baseline_Def10.npy
(108, 56) (108,)
Save 10 to baseline_Def11.npy
Execution Time: 3.102464 seconds
(341913,)
[0 1 2 3 4 5 8 9] [130687      2    665   1113      7 175191  34236     12]
(130687, 56) (130687,)
Save 0 to BIM_Def1.npy
(2, 56) (2,)
Save 1 to BIM_Def2.npy
(665, 56) (665,)
Save 2 to BIM_Def3.npy
(1113, 56) (1113,)
Save 3 to BIM_Def4.npy
(7, 56) (7,)
Save 4 to BIM_Def5.npy
(175191, 56) (175191,)
Save 5 to BIM_Def6.npy
(34236, 56) (34236,)
Save 8 to BIM_Def9.npy
(12, 56) (12,)
Save 

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [47363   429  2010   442   198  4859     1   317 58186    91    75]
(47363, 56) (47363,)
Save 0 to baseline_Def1.npy
(429, 56) (429,)
Save 1 to baseline_Def2.npy
(2010, 56) (2010,)
Save 2 to baseline_Def3.npy
(442, 56) (442,)
Save 3 to baseline_Def4.npy
(198, 56) (198,)
Save 4 to baseline_Def5.npy
(4859, 56) (4859,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(317, 56) (317,)
Save 7 to baseline_Def8.npy
(58186, 56) (58186,)
Save 8 to baseline_Def9.npy
(91, 56) (91,)
Save 9 to baseline_Def10.npy
(75, 56) (75,)
Save 10 to baseline_Def11.npy
Execution Time: 0.170887 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [160264   3358   4270   6299   3513 123108   2606   4989  32398    749
    359]
(160264, 56) (160264,)
Save 0 to BIM_Def1.npy
(3358, 56) (3358,)
Save 1 to BIM_Def2.npy
(4270, 56) (4270,)
Save 2 to BIM_Def3.npy
(6299, 56) (6299,)
Save 3 to BIM_Def4.npy
(3513, 56) (3513,)
Save 4 to BIM_Def5.npy
(123108, 56) 

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}100")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}/UNSW_Input100/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input100.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB100,22326,149,2492,856,66,2019,1,211,85586,201,64,113971
1,XGB100,109193,155,1475,1223,451,175150,1,119,54020,26,100,341913
2,XGB100,94773,80,1387,1225,114,189248,1,732,54285,16,52,341913
3,XGB100,109193,155,1475,1223,451,175150,1,119,54020,26,100,341913
4,XGB100,73977,687,2392,363,199,243611,364,18141,1729,318,132,341913
5,XGB100,126724,365,4746,2543,479,172267,1,576,33703,321,188,341913
6,XGB100,62660,436,7353,2570,196,5779,5,629,261566,530,189,341913
7,XGB100,246800,102,650,1171,170,18560,2,171,72106,1731,450,341913
8,XGB100,114696,354,679,1014,269,174024,0,119,50703,29,26,341913
9,XGB100,110669,80,1473,1274,326,178307,1,279,49428,21,55,341913
